In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from boolrank import DualSiglip2Model
from my_processing import paths_to_dataset

# Load data
data_path = "training"
paths = [
    f"data/{data_path}.jsonl",
    "data/TAR_data.jsonl",
    "data/sysrev_conv.jsonl",
]
dataset = paths_to_dataset(
    paths,
    test_only_sources=['TAR', 'sysrev'],
    # train_sources=['pubmed-searchrefiner','pubmed-query','raw-jsonl']
    train_sources=['pubmed-searchrefiner','raw-jsonl']
)

data/training.jsonl
data/TAR_data.jsonl
data/sysrev_conv.jsonl


Finding similar: 100%|██████████| 40/40 [00:00<00:00, 39747.02it/s]


In [ ]:
N = 1000
df = pd.DataFrame({
    "nl": dataset["train"]["nl_query"],
    "bool": dataset["train"]["bool_query"],
    "quality": dataset["train"]["quality"],
    "source": dataset["train"]["source"]
})
df = df.sample(min(N, df.shape[0])).reset_index(drop=True)
df

(3782, 4)


,nl,bool,quality,source
0,Education for contraceptive use by women after...,"(""Contraception""[Mesh] OR ""Contraception Beha...",0.274709,raw-jsonl
1,Preprocedural mouth rinses for preventing tran...,((dental or dentist$ or hygienist$).mp. AND (...,0.549417,raw-jsonl
2,Clinical efficacy of home-use blue-light thera...,"((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Pimpl...",0.013326,pubmed-searchrefiner
3,Randomized trial of three phototherapy methods...,"((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Vulga...",0.010352,pubmed-searchrefiner
4,Development and implementation of an emergency...,"(""drug overdose""[Mesh] OR Poisoning[Mesh] OR ""...",0.013469,pubmed-searchrefiner
...,...,...,...,...
3777,Treat-to-target in systemic lupus erythematosu...,"((""systemic lupus erythematosus""[tw]) AND (LLD...",0.043195,pubmed-searchrefiner
3778,Sensitivity and specificity of polysomnographi...,(polysomnography[tiab] OR polysomnographic[tia...,0.125265,pubmed-searchrefiner
3779,Controlled Interventions to Reduce Burnout in ...,"(""healthcare staff""[Title/Abstract] OR ""health...",0.062633,pubmed-searchrefiner
3780,Subepidermal moisture predicts erythema and st...,"(""Pressure Ulcer""[Majr] OR ""pressure ulcer*""[T...",0.250530,pubmed-searchrefiner


In [ ]:
model = DualSiglip2Model('BAAI/bge-small-en-v1.5', loss_type="clip")

embeddings = model.encode_text(df["nl"].tolist()).detach().cpu().numpy()

tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            n_iter=1000,
            random_state=42,
            init="pca")
coords = tsne.fit_transform(embeddings)

df["x"] = coords[:,0]
df["y"] = coords[:,1]

fig = px.scatter(
    df, x="x", y="y",
    color="source",               # color by data source
    # size="quality",               # optionally size by quality score
    hover_data={
        "nl": True,               # show the NL query on hover
        "bool": True,             # show the bool query
        "quality": True,
        "x": False, "y": False
    },
    title=f"t-SNE of {N} Text Embeddings"
)

fig.update_traces(marker=dict(opacity=0.7, line_width=0.5))
fig.update_layout(width=800, height=600)
fig.show()